**Dataset:**
personality type data (Lab 2 - Personality Profile Type.csv)

**Objective:**
classify Personality type as one of the following **using RNNs**. <br>
['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']

**Evaluation metric:**
Precision

### Import used libraries

In [61]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import precision_score, classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import string
import keras
import spacy
import re

In [62]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [63]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [64]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

### Load Dataset

In [65]:
df = pd.read_csv("/content/Lab 2 - Personality Profile Type.csv")
df

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments https://www.youtube.com/watch?v=iz7lE1g4XM4 sportscenter not top ten plays https://www.youtube.com/watch?v=uCdfze1etec pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8 http://www.youtube.com/watch?v=u8ejam5DP3E On repeat for most of today.|||May the PerC Experience immerse you.|||The last ...
1,ENTP,'I'm finding the lack of me in these posts very alarming.|||Sex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...|||Giving new meaning to 'Game' theory.|||Hello *ENTP Grin* That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.|||This...
2,INTP,"'Good one _____ https://www.youtube.com/watch?v=fHiGbolFFGw|||Of course, to which I say I know; that's my blessing and my curse.|||Does being absolutely positive that you and your best friend could be an amazing couple count? If so, than yes. Or it's more I could be madly in love in case I reconciled my feelings (which at...|||No, I didn't; thank you for a link!|||So-called Ti-Si loop (and it can stem from any current topic/obsession) can be deadly. It's like when you're stuck in your o..."
3,INTJ,"'Dear INTP, I enjoyed our conversation the other day. Esoteric gabbing about the nature of the universe and the idea that every rule and social code being arbitrary constructs created...|||Dear ENTJ sub, Long time no see. Sincerely, Alpha|||None of them. All other types hurt in deep existential ways that I want no part of.|||Probably a sliding scale that depends on individual preferences, like everything in humanity.|||Draco Malfoy also. I'd say he's either 358 or 368.|||I'm either 358..."
4,ENTJ,'You're fired.|||That's another silly misconception. That approaching is logically is going to be the key to unlocking whatever it is you think you are entitled to. Nobody wants to be approached with BS...|||But guys... he REALLY wants to go on a super-duper-long-ass vacation. C'mon guys. His boss just doesn't listen or get it. He even approached him logically and everything.|||Never mind. Just go on permanent vacation.|||Two months? I wouldn't be crazy about the idea. If you are really hi...
...,...,...
8670,ISFP,"'https://www.youtube.com/watch?v=t8edHB_h908|||IxFP just because I always think of cats as Fi doms for some reason. https://www.youtube.com/watch?v=ib28iGc9go4|||Especially on websites that have become a haven for neo nazis, such as PerC.|||I'm such a nerd. I've been learning Dutch on Duolingo and I'm having so much fun :)|||Duolingo is the shit! Oh my god I love this XD|||Right wingers do this because they lack political consciousness.|||I doubt it, but if they are real then I hope to he..."
8671,ENFP,"'So...if this thread already exists someplace else (which it does: http://personalitycafe.com/enfp-forum-inspirers/116902-movies-loved-enfp.html#post2952851), how the heck to I delete this one here???|||Ooops...I guess I didn't look too hard because I just started a Movies every ENFP should watch thread...|||So, this came up in the You Know You're An ENFP When... thread. (@Pucca : here we go! IT IS DONE :D) This isn't a list of films with ENFPs IN them, but rather ones that you believe E..."
8672,INTP,'So many questions when i do these things. I would take the purple pill. Pick winning lottery numbers. Do whatever I want.|||I worked in a brewery for a couple years here. I dont know if you're familiar with the local micros/craft at all but Mill st. was the name of it. Their tankhouse is my favorite of anything local.|||I am going to summon the great and powerful mr cafebot and his song of destruction. The four horseman ride. 

### Data splitting

In [66]:
x = df[["posts"]]
y = df["type"]
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [67]:
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

X_train (6940, 1)
X_test (1735, 1)
y_train (6940,)
y_test (1735,)


### EDA on training data

- check NaNs

In [68]:
nan_counts = df.isnull().sum()
print(nan_counts)

type     0
posts    0
dtype: int64


- check duplicates

In [69]:
duplicates = df["posts"].duplicated()
num_duplicates = duplicates.sum()
print("Number of duplicate rows:", num_duplicates)

Number of duplicate rows: 0


- show a representative sample of data texts to find out required preprocessing steps

In [70]:
sample = df['posts'].sample(5, random_state=42)
print(sample)

2802    'This. When I lie it's to avoid an unreasonable response for something I did that was completely justifiable to me but if I was to try to justify to someone else they'd be too hung up on what I did. I...|||Your IQ (SD15): 128|||I typically have very difficult times maintaining friends I've come to realize. I've had people I talk to for years and I still don't want to talk to them... I DID, but then I just got bored of them,...|||An INTJ friend of mine told me she really hates eating because ...
2166    'I said zero flexibility and little time for dating. Going around sifting through dates to find one that clicks isn't going to be as viable.  That has nothing to do with maintaining a relationship I...|||How you've come to the conclusion I have nothing to offer  merely because I am busy is beyond me.       No not really. Again, more made up stuff.|||okay, this isn't cool.  you're specifically looking to get into a relationship because you're about to enter a life phase where you 

- check dataset balancing

In [71]:
class_distribution = df['type'].value_counts()
print(class_distribution)

type
INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: count, dtype: int64


## Cleaning and Preprocessing

In [72]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass


    def fit(self, X, y=None):
        pass


    def clean_data(self ,text):
        url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        text = re.sub(url_pattern, '', text)
        emoji_pattern = re.compile(r'[\U00010000-\U0010ffff]')
        text = re.sub(emoji_pattern, '', text)
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        return text

    def transform(self, X):

        transformed_X = X.copy()

        transformed_X['posts'] = transformed_X['posts'].str.lower()
        transformed_X['posts'] = transformed_X['posts'].apply(self.clean_data)

        return transformed_X["posts"].values


    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X)


In [73]:
# Preprocessing for X_train
custom_transformer = CustomTransformer()
transformed_df = custom_transformer.fit_transform(X_train)

In [74]:
# Preprocessing for X_test
custom_transformer_test = CustomTransformer()
transformed_df_test = custom_transformer_test.fit_transform(X_test)

In [75]:
transformed_df[0:1]

array(['i loved all the light we cannot see by anthony doerr its a pretty recent book came out in may but it was simply amazingthe strange and beautiful sorrows of ava lavender  leslye waltonmy family would want me to save the  billion people i would make the same choice it would be horrible either wayi dont put much stock in numerology but i figured it couldnt hurt to try and it seemed like fun  anyway   birth force number   destiny number   personality number   soul urgedoctor who absolute favourite supernatural glee bones csi the voice of holland big bang theory death note and other animes  mythbusters derren brown showshmm i was bulllied from  to  it was never anything physical it were comments on my clothes my hair my personality everything basically i was tall and shy so i kinda stood out there was onefantasy is absolutely my favourite gene  i like scifi a lot as well other genres are psychological contemporary sometimes but really i dont stick to one particular genre i prefer bo

In [76]:
tokens = Tokenizer()
tokens.fit_on_texts(transformed_df)
train_sequences = tokens.texts_to_sequences(transformed_df)
test_sequences = tokens.texts_to_sequences(transformed_df_test)

In [77]:
len(train_sequences)

6940

In [78]:
list_size_seq = [len(s) for s in train_sequences]
max(list_size_seq)

1875

## Padding

In [79]:
x_train_padding = pad_sequences(train_sequences, maxlen=900,padding='post')
print("x_train_padding",x_train_padding.shape)

x_train_padding (6940, 900)


In [80]:
x_test_padded = pad_sequences(test_sequences, maxlen=900, padding='post')
print("x_test_padded",x_test_padded.shape)

x_test_padded (1735, 900)


## One Hot Encoding For Categorical Data

In [81]:
Encoder = LabelEncoder()

y_encode = Encoder.fit_transform(y_train)
y_train_encode = tf.keras.utils.to_categorical(y_encode)

x_encode = Encoder.fit_transform(y_test)
y_test_encode =tf.keras.utils.to_categorical(x_encode)

print("y_train_encode",y_train_encode.shape)
print("y_test_encode",y_test_encode.shape)

y_train_encode (6940, 16)
y_test_encode (1735, 16)


## Modelling

In [82]:
VOCAB_SIZE = len(tokens.word_index)+1

In [83]:
X_validation, X_test_final, y_validation, y_test_final = train_test_split(x_test_padded,y_test_encode, test_size=0.3, random_state=42)
print("X_validation",X_validation.shape)
print("X_test_final",X_test_final.shape)
print("y_validation",y_validation.shape)
print("y_test_final",y_test_final.shape)

X_validation (1214, 900)
X_test_final (521, 900)
y_validation (1214, 16)
y_test_final (521, 16)


# RNN Model

In [39]:
Rnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 128),
    tf.keras.layers.SimpleRNN(128),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(16, activation='softmax')
])

In [40]:
Rnn_model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [41]:
history = Rnn_model.fit(x_train_padding, y_train_encode, epochs=5, batch_size=128,
                    validation_data=(X_validation, y_validation),
                    validation_steps=30)

Epoch 1/5
55/55 [==============================] - ETA: 0s - loss: 2.3340 - accuracy: 0.1994

55/55 [==============================] - 158s 3s/step - loss: 2.3340 - accuracy: 0.1994 - val_loss: 2.2867 - val_accuracy: 0.2150
Epoch 2/5
55/55 [==============================] - 142s 3s/step - loss: 2.2648 - accuracy: 0.2157
Epoch 3/5
55/55 [==============================] - 150s 3s/step - loss: 2.2832 - accuracy: 0.2265
Epoch 4/5
55/55 [==============================] - 140s 3s/step - loss: 2.2633 - accuracy: 0.2229
Epoch 5/5
55/55 [==============================] - 142s 3s/step - loss: 2.2005 - accuracy: 0.2736


### **Evaluation metric for RNN Model:**
Precision

In [55]:
test_loss, test_acc = Rnn_model.evaluate(X_test_final, y_test_final)

y_pred_final = Rnn_model.predict(X_test_final)
y_pred = np.argmax(y_pred_final, axis=1)
y_test = np.argmax(y_test_final, axis=1)

precision = precision_score(y_test, y_pred, average="macro")
precision_micro = precision_score(y_test, y_pred, average="micro")
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))
print(f'Macro-averaged Precision: {precision:.2f}')
print(f'Micro-averaged Precision: {precision_micro:.2f}')

17/17 [==============================] - 3s 181ms/step
Test Loss: 2.327441453933716
Test Accuracy: 0.1785028725862503
Macro-averaged Precision: 0.04
Micro-averaged Precision: 0.18


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# GRU Model

In [91]:
gru_bi = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 128),
    tf.keras.layers.GRU(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='softmax')
])

In [92]:
gru_bi.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [93]:
history = gru_bi.fit(x_train_padding, y_train_encode, epochs=5, batch_size=128,
                    validation_data=(X_validation, y_validation),
                    validation_steps=30)

Epoch 1/5
55/55 [==============================] - ETA: 0s - loss: 2.4913 - accuracy: 0.2016

55/55 [==============================] - 163s 3s/step - loss: 2.4913 - accuracy: 0.2016 - val_loss: 2.2808 - val_accuracy: 0.2150
Epoch 2/5
55/55 [==============================] - 154s 3s/step - loss: 2.2593 - accuracy: 0.2108
Epoch 3/5
55/55 [==============================] - 153s 3s/step - loss: 2.1623 - accuracy: 0.2337
Epoch 4/5
55/55 [==============================] - 154s 3s/step - loss: 1.9160 - accuracy: 0.3429
Epoch 5/5
55/55 [==============================] - 150s 3s/step - loss: 1.6043 - accuracy: 0.4670


### **Evaluation metric GRU Model:**
Precision

In [94]:
test_loss, test_acc = gru_bi.evaluate(X_test_final, y_test_final)

y_pred_final = gru_bi.predict(X_test_final)
y_pred = np.argmax(y_pred_final, axis=1)
y_test = np.argmax(y_test_final, axis=1)

precision = precision_score(y_test, y_pred, average="macro")
precision_micro = precision_score(y_test, y_pred, average="micro")
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))
print(f'Macro-averaged Precision: {precision:.2f}')
print(f'Micro-averaged Precision: {precision_micro:.2f}')

17/17 [==============================] - 2s 106ms/step
Test Loss: 3.0659711360931396
Test Accuracy: 0.15738964080810547
Macro-averaged Precision: 0.04
Micro-averaged Precision: 0.16


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# LSTM Model

In [95]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 128),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(16, activation='softmax')
])

In [96]:
lstm_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [97]:
history = lstm_model.fit(x_train_padding, y_train_encode, epochs=5, batch_size=128,
                    validation_data=(X_validation, y_validation),
                    validation_steps=30)

Epoch 1/5
55/55 [==============================] - ETA: 0s - loss: 2.3719 - accuracy: 0.1958

55/55 [==============================] - 264s 5s/step - loss: 2.3719 - accuracy: 0.1958 - val_loss: 2.2717 - val_accuracy: 0.2150
Epoch 2/5
55/55 [==============================] - 241s 4s/step - loss: 2.2800 - accuracy: 0.2081
Epoch 3/5
55/55 [==============================] - 236s 4s/step - loss: 2.2313 - accuracy: 0.2229
Epoch 4/5
55/55 [==============================] - 242s 4s/step - loss: 1.9852 - accuracy: 0.3500
Epoch 5/5
55/55 [==============================] - 247s 4s/step - loss: 1.5340 - accuracy: 0.5017


### **Evaluation metric LSTM Model:**
Precision

In [98]:
test_loss, test_acc = gru_bi.evaluate(X_test_final, y_test_final)
y_pred_final = lstm_model.predict(X_test_final)
y_pred = np.argmax(y_pred_final, axis=1)
y_test = np.argmax(y_test_final, axis=1)

precision = precision_score(y_test, y_pred, average="macro")
precision_micro = precision_score(y_test, y_pred, average="micro")
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))
print(f'Macro-averaged Precision: {precision:.2f}')
print(f'Micro-averaged Precision: {precision_micro:.2f}')

17/17 [==============================] - 8s 422ms/step
Test Loss: 3.0659711360931396
Test Accuracy: 0.15738964080810547
Macro-averaged Precision: 0.06
Micro-averaged Precision: 0.17


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Conclusion and final results


#### Done!